In [1]:
!pip3 install imutils
# 

  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25860 sha256=b2899ea402dc375067e30c84ed329f3e0e5a36c13c84d8bdcef54c23a7be58e4
  Stored in directory: /root/.cache/pip/wheels/86/d7/0a/4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
Successfully built imutils


In [2]:
import os
import cv2
from tqdm import tqdm
from scipy.spatial import distance as dist
import imutils
import numpy as np
from sklearn.utils import shuffle
from sklearn import decomposition
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

## Loading Data, edge detection  and thresholding

In [3]:
def findEdges(image):
    # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(image, (1, 1), 0)
    edged = cv2.Canny(gray, 100, 400)
    edged = cv2.dilate(edged, None, iterations=1)
    edged = cv2.erode(edged, None, iterations=1)
    return edged


In [4]:
def getImgContours(edged):
    # find contours in the edge map
    contours = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    contours = sorted(contours, key=lambda x: cv2.contourArea(x))
    return contours



In [5]:
def getBoxes(contours, orig):
    boxes = []
    centers = []
    for contour in contours:
        box = cv2.minAreaRect(contour)
        box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
        box = np.array(box, dtype="int")
        (tl, tr, br, bl) = box
        if (dist.euclidean(tl, bl)) > 0 and (dist.euclidean(tl, tr)) > 0:
            boxes.append(box)
    return boxes


In [6]:
class_names = ['EOSINOPHIL', 'LYMPHOCYTE', 'MONOCYTE', 'NEUTROPHIL']
nb_classes = len(class_names)
image_size = (80,80)

In [7]:

def load_data():
    
    datasets = ['/kaggle/input/blood-cells/dataset2-master/dataset2-master/images/TRAIN',
                '/kaggle/input/blood-cells/dataset2-master/dataset2-master/images/TEST' ]    
    images = []
    labels = []

    # iterate through training and test sets
    count=0
    for dataset in datasets:

        # iterate through folders in each dataset
        
        for folder in os.listdir(dataset):

            if folder in ['EOSINOPHIL']: label = 1#[1, 0, 0, 0]
            elif folder in ['LYMPHOCYTE']: label = 2#[0, 1, 0, 0]
            elif folder in ['MONOCYTE']: label = 3#[0, 0, 1, 0]
            elif folder in ['NEUTROPHIL']: label = 4#[0, 0, 0, 1]

            # iterate through each image in folder
           
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):

                # get pathname of each image
                img_path = os.path.join(os.path.join(dataset, folder), file)

                # Open and resize the| img
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.copyMakeBorder(image,10,10,10,10,cv2.BORDER_CONSTANT,value=[198, 203, 208])
                image1 = cv2.inRange(image,(80, 80, 180),(180, 170, 245))
                kernel = np.ones((3, 3), np.uint8)
                kernel1 = np.ones((5, 5), np.uint8)
                img_erosion = cv2.erode(image1, kernel, iterations=2)
                image1 = cv2.dilate(img_erosion, kernel1, iterations=4)
                edgedImage = findEdges(image1)
                edgedContours = getImgContours(edgedImage)
                edgedBoxes =  getBoxes(edgedContours, image.copy())
                if len(edgedBoxes)==0:
                    count +=1
                    continue
                last = edgedBoxes[-1]
                max_x = int(max(last[:,0]))
                min_x = int( min(last[:,0]))
                max_y = int(max(last[:,1]))
                min_y = int(min(last[:,1]))
                mask = np.zeros_like(image)
                cv2.drawContours(mask, edgedContours, len(edgedContours)-1, (255,255,255), -1) 
                image[mask==0] = 0
                image = image[min_y:max_y, min_x:max_x]

                if (np.size(image)==0):
                    count +=1
                    continue
#                 image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
#                 plt.imshow(image)  
#                 plt.show()
                image = cv2.resize(image, image_size)
                size  = np.size(image)
                image = np.reshape(image,size )
                
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
    print(count)
    images = np.array(images, dtype = 'uint8')
    labels = np.array(labels, dtype = 'uint8')

    return images, labels, size

In [8]:
images, labels, size = load_data()

100%|██████████| 623/623 [00:05<00:00, 103.89it/s]


303


## Scaling data

In [9]:
scaler = MinMaxScaler(copy=False ,feature_range=(-1,1))
images = scaler.fit_transform(images)


In [10]:
np.max(images)

1.0

In [11]:
images = np.append(np.ones((images.shape[0],1)),images,axis=1)

In [12]:

images, labels = shuffle(images, labels, random_state=10)
# images = images[0:6000] ,labels = labels[0:6000], n = int(np.shape(images)[0]*0.8)
print(" 00")
# train_images, test_images = images[indeces[0:n]], images[indeces[n:]] , train_labels, test_labels = labels[indeces[0:n]], labels[indeces[n:]]
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size = 0.2)
print(" 00")

 00
 00


In [13]:
del images ,labels

In [14]:
np.shape(train_images)

(9712, 19201)

In [15]:
test_labels[0].size

1

# Classification

    we chose Random Forest and Support Vector machine as they can deal with great number of features.

### 1. Random Forest

In [16]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=30, random_state=0)
clf.fit(train_images,train_labels)
print(clf.score(test_images,test_labels))

0.9209551255660766


In [17]:

pred = clf.predict(test_images)
pred2 = clf.predict(train_images)


In [18]:
test_labels

array([4, 4, 1, ..., 2, 4, 1], dtype=uint8)

## Getting Accuracy Score

In [19]:
from sklearn.metrics import accuracy_score
print("Test accuracy",accuracy_score(test_labels, pred))
print("Train accuracy",accuracy_score(train_labels,pred2))


Test accuracy 0.9209551255660766
Train accuracy 1.0
